In [1]:
import torch
import torch.nn as nn
import os
from ultralytics import YOLO
import cv2
from roboflow import Roboflow

In [6]:
# set up directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
parent_dir = os.path.dirname(parent_dir)
ckpt_path = os.path.join(current_dir+"\\ckpt\\helmet-detect-yolov8.pt")
input_path = os.path.join(parent_dir+"\\data\\input\\")
output_path = os.path.join(parent_dir+"\\data\\output\\")
input_path, output_path

('d:\\VSC\\yolov8-detect\\data\\input\\',
 'd:\\VSC\\yolov8-detect\\data\\output\\')

In [7]:
# Set up video capture and writer
cap = cv2.VideoCapture(input_path+"input.mp4")
out = cv2.VideoWriter(output_path+'output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (640, 480))

In [8]:
# follow https://github.com/janahmorano/Helmet-Detection-Model/tree/main
# load checkpoint
model = YOLO(ckpt_path)

In [9]:
# set model to evaluation mode
# model.eval()
results = model(source=input_path+"input.mp4",stream=True)

# Process video frames
for r in results:
    frame = r.orig_img  # Directly use the original image from results
    if frame is None:
        break

    # Draw bounding boxes on the frame
    for box in r.boxes:
        x1, y1, x2, y2 = box.xyxy[0].int().tolist()  # Convert to int for pixel coordinates
        # get classID
        class_id = box.cls[0].item() 
        
        if class_id==0:
            color=(0,255,0)
        elif class_id==1:
            color=(0,0,255)

        # draw rounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    # Ensure the frame is of correct size
    frame = cv2.resize(frame, (640, 480))  # Resize if necessary
    out.write(frame)  # Write the processed frame to output video

# Release resources
cap.release()
out.release()


video 1/1 (frame 1/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 4 helmets, 46.8ms
video 1/1 (frame 2/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 8 helmets, 32.5ms
video 1/1 (frame 3/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 5 helmets, 33.7ms
video 1/1 (frame 4/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 5 helmets, 1 no_helmet, 24.1ms
video 1/1 (frame 5/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 6 helmets, 23.6ms
video 1/1 (frame 6/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 5 helmets, 23.2ms
video 1/1 (frame 7/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 7 helmets, 24.1ms
video 1/1 (frame 8/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 4 helmets, 24.9ms
video 1/1 (frame 9/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 5 helmets, 20.1ms
video 1/1 (frame 10/224) d:\VSC\yolov8-detect\data\input\input.mp4: 256x416 6 helmets, 23.7ms
video 1/1 (frame 11/224) d:\VSC\yolov8-detect\data\inpu

In [5]:
# Set up Roboflow with your API key
rf = Roboflow(api_key="ujW2M3sEr0QJJO2W4YIC")  # Replace with your actual API key
project = rf.workspace().project("helmet-detection-yolov8")  # Replace with your project name
model = project.version(1).model

# Set up video capture and writer
cap = cv2.VideoCapture(input_path+"movie2.mp4")
out = cv2.VideoWriter(output_path+'output_movie2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (640, 480))

# Process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform inference using the model from Roboflow
    prediction = model.predict(frame, confidence=40)  # Confidence threshold of 40%
    predictions = prediction.json()['predictions']

    # Draw bounding boxes on the frame
    for pred in predictions:
        x1, y1 = int(pred['x'] - pred['width'] / 2), int(pred['y'] - pred['height'] / 2)
        x2, y2 = int(pred['x'] + pred['width'] / 2), int(pred['y'] + pred['height'] / 2)
        
        # Determine color based on label (assuming "helmet" and "no_helmet")
        label = pred['class']
        if label == "helmet":
            color = (0, 255, 0)  # Green for helmet
        elif label == "no_helmet":
            color = (0, 0, 255)  # Red for no helmet
        else:
            color = (255, 255, 255)  # White for unknown class

        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Resize frame to 640x480 if necessary
    frame = cv2.resize(frame, (640, 480))
    out.write(frame)

# Release resources
cap.release()
out.release()


loading Roboflow workspace...
loading Roboflow project...
